# GOOGLE COLAB NOTEBOOK

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import torch
from torch import nn
from torch.utils.data import DataLoader
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import torch.nn.functional as F
from src.main import convert_to_one_hot
from src.main import prepare_attribut_data, prepare_result_data
from src.main import DRAW, HOME_WINS, AWAY_WINS

In [ ]:
learning_rate=0.01
n_epochs=10
batch_size=50
n_components = 280
criterion=nn.CrossEntropyLoss()
# Type either pca or kpca or None
type_pca=None
kpca_kernel='poly'

columns=['ID','TEAM_SUCCESSFUL_PASSES_5_last_match_average','TEAM_BALL_POSSESSION_5_last_match_average','TEAM_DANGEROUS_ATTACKS_5_last_match_average'
,'TEAM_GAME_WON_season_sum','TEAM_GAME_DRAW_season_sum','TEAM_GAME_LOST_season_sum','TEAM_GOALS_season_sum']
columns=[]


In [ ]:


train_path_away = '/content/drive/MyDrive/data/Train_Data/train_away_team_statistics_df.csv'
train_path_home = '/content/drive/MyDrive/data/Train_Data/train_home_team_statistics_df.csv'
train_path_result = '/content/drive/MyDrive/data/Train_Data/Y_train.csv'
df_attribut= prepare_attribut_data(train_path_away, train_path_home,columns,use_player_data=True).drop(['ID'], axis=1)
df_result=pd.read_csv('/content/drive/MyDrive/data/Train_Data/Y_train.csv')
df_result_final=df_result.drop(['ID'], axis=1)
df_result.head()

,ID,HOME_WINS,DRAW,AWAY_WINS
0,0,0,0,1
1,1,0,1,0
2,2,0,0,1
3,3,1,0,0
4,4,0,1,0


In [ ]:
df_attribut.head()

,HOME_TEAM_SHOTS_TOTAL_season_sum,HOME_TEAM_SHOTS_INSIDEBOX_season_sum,HOME_TEAM_SHOTS_OFF_TARGET_season_sum,HOME_TEAM_SHOTS_ON_TARGET_season_sum,HOME_TEAM_SHOTS_OUTSIDEBOX_season_sum,HOME_TEAM_PASSES_season_sum,HOME_TEAM_SUCCESSFUL_PASSES_season_sum,HOME_TEAM_SAVES_season_sum,HOME_TEAM_CORNERS_season_sum,HOME_TEAM_FOULS_season_sum,...,HOME_PLAYER_STARTING_LINEUP_5_last_match_std,HOME_PLAYER_SUCCESSFUL_DRIBBLES_5_last_match_std,HOME_PLAYER_TACKLES_5_last_match_std,HOME_PLAYER_TOTAL_CROSSES_5_last_match_std,HOME_PLAYER_TOTAL_DUELS_5_last_match_std,HOME_PLAYER_YELLOWCARDS_5_last_match_std,HOME_PLAYER_PUNCHES_5_last_match_std,HOME_PLAYER_LONG_BALLS_5_last_match_std,HOME_PLAYER_LONG_BALLS_WON_5_last_match_std,HOME_PLAYER_SHOTS_OFF_TARGET_5_last_match_std
0,3.0,2.0,5.0,2.0,1.0,2.0,2.0,5.0,3.0,6.0,...,623.0,409.0,431.0,250.0,666.0,710.0,0.0,0.0,0.0,0.0
1,6.0,8.0,3.0,6.0,5.0,8.0,7.0,10.0,6.0,8.0,...,63.0,180.0,245.0,156.0,397.0,357.0,0.0,0.0,0.0,0.0
2,4.0,2.0,5.0,2.0,8.0,1.0,1.0,2.0,2.0,7.0,...,1180.0,286.0,527.0,252.0,774.0,805.0,0.0,0.0,0.0,0.0
3,7.0,5.0,5.0,6.0,6.0,9.0,9.0,2.0,2.0,0.0,...,357.0,159.0,344.0,307.0,353.0,280.0,0.0,0.0,0.0,0.0
4,3.0,3.0,2.0,3.0,4.0,4.0,3.0,4.0,4.0,7.0,...,329.0,327.0,414.0,262.0,376.0,248.0,0.0,0.0,0.0,0.0


In [ ]:
df_attribut.head()

,HOME_TEAM_SHOTS_TOTAL_season_sum,HOME_TEAM_SHOTS_INSIDEBOX_season_sum,HOME_TEAM_SHOTS_OFF_TARGET_season_sum,HOME_TEAM_SHOTS_ON_TARGET_season_sum,HOME_TEAM_SHOTS_OUTSIDEBOX_season_sum,HOME_TEAM_PASSES_season_sum,HOME_TEAM_SUCCESSFUL_PASSES_season_sum,HOME_TEAM_SAVES_season_sum,HOME_TEAM_CORNERS_season_sum,HOME_TEAM_FOULS_season_sum,...,HOME_PLAYER_STARTING_LINEUP_5_last_match_std,HOME_PLAYER_SUCCESSFUL_DRIBBLES_5_last_match_std,HOME_PLAYER_TACKLES_5_last_match_std,HOME_PLAYER_TOTAL_CROSSES_5_last_match_std,HOME_PLAYER_TOTAL_DUELS_5_last_match_std,HOME_PLAYER_YELLOWCARDS_5_last_match_std,HOME_PLAYER_PUNCHES_5_last_match_std,HOME_PLAYER_LONG_BALLS_5_last_match_std,HOME_PLAYER_LONG_BALLS_WON_5_last_match_std,HOME_PLAYER_SHOTS_OFF_TARGET_5_last_match_std
0,3.0,2.0,5.0,2.0,1.0,2.0,2.0,5.0,3.0,6.0,...,623.0,409.0,431.0,250.0,666.0,710.0,0.0,0.0,0.0,0.0
1,6.0,8.0,3.0,6.0,5.0,8.0,7.0,10.0,6.0,8.0,...,63.0,180.0,245.0,156.0,397.0,357.0,0.0,0.0,0.0,0.0
2,4.0,2.0,5.0,2.0,8.0,1.0,1.0,2.0,2.0,7.0,...,1180.0,286.0,527.0,252.0,774.0,805.0,0.0,0.0,0.0,0.0
3,7.0,5.0,5.0,6.0,6.0,9.0,9.0,2.0,2.0,0.0,...,357.0,159.0,344.0,307.0,353.0,280.0,0.0,0.0,0.0,0.0
4,3.0,3.0,2.0,3.0,4.0,4.0,3.0,4.0,4.0,7.0,...,329.0,327.0,414.0,262.0,376.0,248.0,0.0,0.0,0.0,0.0


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df_attribut, df_result_final, test_size=0.1, random_state=10)

In [ ]:
from sklearn.decomposition import PCA
from sklearn.decomposition import KernelPCA

if type_pca=='PCA':
  pca_version = PCA(n_components=n_components)
elif type_pca=='KPCA':
  pca_version = KernelPCA(n_components=n_components, kernel=kpca_kernel)
else:
  pca_version = None

if pca_version is not None:
    pca_version.fit(X_train)
    X_train = pd.DataFrame(pca_version.transform(X_train), index=X_train.index, columns=["PC" + str(i) for i in range(1, n_components + 1)])
    X_test = pd.DataFrame(pca_version.transform(X_test), index=X_test.index, columns=["PC" + str(i) for i in range(1, n_components + 1)])

In [ ]:
X_train

,HOME_TEAM_SHOTS_TOTAL_season_sum,HOME_TEAM_SHOTS_INSIDEBOX_season_sum,HOME_TEAM_SHOTS_OFF_TARGET_season_sum,HOME_TEAM_SHOTS_ON_TARGET_season_sum,HOME_TEAM_SHOTS_OUTSIDEBOX_season_sum,HOME_TEAM_PASSES_season_sum,HOME_TEAM_SUCCESSFUL_PASSES_season_sum,HOME_TEAM_SAVES_season_sum,HOME_TEAM_CORNERS_season_sum,HOME_TEAM_FOULS_season_sum,...,HOME_PLAYER_STARTING_LINEUP_5_last_match_std,HOME_PLAYER_SUCCESSFUL_DRIBBLES_5_last_match_std,HOME_PLAYER_TACKLES_5_last_match_std,HOME_PLAYER_TOTAL_CROSSES_5_last_match_std,HOME_PLAYER_TOTAL_DUELS_5_last_match_std,HOME_PLAYER_YELLOWCARDS_5_last_match_std,HOME_PLAYER_PUNCHES_5_last_match_std,HOME_PLAYER_LONG_BALLS_5_last_match_std,HOME_PLAYER_LONG_BALLS_WON_5_last_match_std,HOME_PLAYER_SHOTS_OFF_TARGET_5_last_match_std
9267,1.0,1.0,0.0,2.0,5.0,0.0,0.0,8.0,3.0,0.0,...,686.0,258.0,380.0,234.0,483.0,322.0,99.0,0.0,0.0,0.0
10281,8.0,7.0,9.0,5.0,6.0,3.0,3.0,5.0,6.0,10.0,...,981.0,313.0,536.0,169.0,642.0,1092.0,0.0,0.0,0.0,0.0
1044,7.0,7.0,7.0,7.0,7.0,9.0,8.0,0.0,7.0,4.0,...,777.0,229.0,353.0,315.0,391.0,402.0,0.0,0.0,0.0,0.0
3914,8.0,10.0,5.0,10.0,4.0,9.0,9.0,5.0,7.0,7.0,...,126.0,428.0,518.0,149.0,440.0,331.0,34.0,0.0,0.0,0.0
11168,4.0,6.0,4.0,7.0,1.0,6.0,5.0,10.0,5.0,7.0,...,392.0,216.0,552.0,175.0,363.0,423.0,66.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7291,6.0,4.0,5.0,8.0,8.0,6.0,6.0,4.0,3.0,5.0,...,406.0,448.0,425.0,240.0,585.0,609.0,0.0,0.0,0.0,0.0
11633,6.0,7.0,10.0,2.0,5.0,3.0,3.0,0.0,9.0,4.0,...,644.0,173.0,368.0,225.0,612.0,595.0,0.0,0.0,0.0,0.0
1344,9.0,9.0,8.0,7.0,7.0,8.0,8.0,2.0,7.0,9.0,...,714.0,278.0,470.0,466.0,462.0,411.0,30.0,0.0,0.0,0.0
7293,1.0,2.0,1.0,2.0,0.0,1.0,0.0,7.0,1.0,10.0,...,956.0,354.0,553.0,274.0,802.0,375.0,97.0,0.0,0.0,0.0


In [ ]:
if type_pca=='PCA':
    import seaborn as sns
    sns.set_style('darkgrid')
    import operator

    import matplotlib.pyplot as plt
    plt.figure(figsize=(16, 10))
    sns.barplot(x=pca_version.explained_variance_ratio_, y=["PC" + str(i) for i in range(1, n_components + 1)], orient='h', palette='husl')
    plt.xlim(0., 1.)
    plt.xlabel("Proportion of Variance in Original Data")
    plt.title("Principal Component Variance")
    plt.show()


In [ ]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [ ]:
device

device(type='cuda')

In [ ]:
from torch.utils.data import Dataset, DataLoader
class Data(Dataset):
    def __init__(self):
        self.x=torch.from_numpy(np.array(X_train)).to(torch.float32)
        self.y=torch.from_numpy(np.array(y_train)).to(torch.long)
        self.len=self.x.shape[0]
    def __getitem__(self,index):
        return self.x[index], self.y[index]
    def __len__(self):
        return self.len

In [ ]:
def get_layer_size(init_length):
   layers_size=[init_length]
   while init_length/2>3:
      layers_size.append(init_length//2)
      init_length=init_length//2
   layers_size.append(3)
   return layers_size

def get_layers_from_list(layer_size,activation,p_dropout):
  layers=[]
  for i in range(len(layer_size)-1):
      layers.append(nn.Linear(layer_size[i],layer_size[i+1]))
      layers.append(activation)
      layers.append( nn.Dropout(p_dropout))

  return layers


In [ ]:


# class MLP(nn.Module):
#     def __init__(self):
#         super().__init__()
#         self.feature_size=len(X_train.columns)
#         self.activ=nn.ReLU()
#         self.p=0
#         layer_list=get_layers_from_list(get_layer_size(self.feature_size),self.activ,self.p)
#         self.layers = nn.Sequential(
#             *layer_list,
#             nn.Softmax(),
#         )

#     def forward(self, x):
#         return self.layers(x)

In [ ]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.activ=nn.ReLU()
        self.p=0
        self.layers = nn.Sequential(
            nn.Linear(884, 50),
            self.activ,
            nn.Linear(50, 25),
            self.activ,
            nn.Linear(25, 3),
            nn.Softmax(),
        )

    def forward(self, x):
        return self.layers(x)

In [ ]:
data_set=Data()
model = MLP()
model=model.to(device)

In [ ]:
model.layers[0].weight.dtype

torch.float32

In [ ]:
criterion=criterion.to(device)

In [ ]:
trainloader=DataLoader(dataset=data_set,batch_size=batch_size)
dataloader_iterator = iter(trainloader)
x, y = next(dataloader_iterator)

In [ ]:
model(x.to(device))

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py:1553: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


tensor([[4.4833e-36, 1.0000e+00, 4.2715e-17],
        [1.2950e-28, 1.0000e+00, 7.2937e-14],
        [6.5128e-34, 1.0000e+00, 4.1001e-21],
        [1.7214e-37, 1.0000e+00, 2.5590e-16],
        [2.0394e-31, 1.0000e+00, 5.6386e-17],
        [1.6053e-30, 1.0000e+00, 1.4519e-14],
        [3.8403e-27, 1.0000e+00, 1.6397e-11],
        [3.1511e-29, 1.0000e+00, 4.3988e-12],
        [1.1784e-32, 1.0000e+00, 3.7644e-17],
        [4.0088e-37, 1.0000e+00, 3.0579e-21],
        [8.7833e-41, 1.0000e+00, 2.8359e-22],
        [2.8358e-40, 1.0000e+00, 3.2817e-23],
        [7.0095e-32, 1.0000e+00, 2.7682e-16],
        [1.8819e-36, 1.0000e+00, 1.5826e-17],
        [8.1530e-07, 1.0000e+00, 2.9181e-07],
        [6.2042e-28, 1.0000e+00, 7.1479e-18],
        [5.4354e-29, 1.0000e+00, 1.1388e-19],
        [3.6498e-29, 1.0000e+00, 1.8610e-14],
        [2.2116e-38, 1.0000e+00, 6.4849e-19],
        [4.3173e-16, 1.0000e+00, 6.4309e-16],
        [4.2744e-32, 1.0000e+00, 7.9803e-15],
        [2.9879e-34, 1.0000e+00, 5

In [ ]:

optimizer=torch.optim.SGD(model.parameters(), lr=learning_rate)

In [ ]:
loss_list=[]

#n_epochs
for epoch in range(n_epochs):
    print(f'epoch {epoch}')
    for x, y in trainloader:


        #clear gradient
        optimizer.zero_grad()
        #make a prediction
        z=model(x.to(device))
        loss=criterion(z,y.to(torch.float32).to(device))
        # calculate gradients of parameters
        loss.backward()
        # update parameters
        optimizer.step()

        loss_list.append(loss.data)
    print(f'loss is {loss_list[-1]}')


epoch 0
loss is 1.324172019958496
epoch 1
loss is 1.324172019958496
epoch 2
loss is 1.324172019958496
epoch 3
loss is 1.324172019958496
epoch 4
loss is 1.324172019958496
epoch 5
loss is 1.324172019958496
epoch 6
loss is 1.324172019958496
epoch 7
loss is 1.324172019958496
epoch 8
loss is 1.324172019958496
epoch 9
loss is 1.324172019958496


In [ ]:
loss_list[-1]

tensor(1.3242, device='cuda:0')

In [ ]:
model.layers[0].weight

Parameter containing:
tensor([[ 0.0229, -0.0292,  0.0136,  ..., -0.0102,  0.0118,  0.0257],
        [-0.0325, -0.0143, -0.0071,  ..., -0.0179,  0.0172,  0.0171],
        [ 0.0148, -0.0131,  0.0286,  ..., -0.0181,  0.0320,  0.0238],
        ...,
        [ 0.0018,  0.0295,  0.0283,  ..., -0.0106,  0.0043, -0.0140],
        [ 0.0150,  0.0188,  0.0303,  ..., -0.0141, -0.0156,  0.0219],
        [-0.0031, -0.0061,  0.0313,  ..., -0.0123, -0.0319, -0.0158]],
       device='cuda:0', requires_grad=True)

In [ ]:
test_data = torch.from_numpy(np.array(X_test)).to(torch.float32).to(device)
test_targets = torch.from_numpy(np.array(y_test)).to(torch.long).to(device)

In [ ]:
model.eval()

MLP(
  (activ): ReLU()
  (layers): Sequential(
    (0): Linear(in_features=884, out_features=50, bias=True)
    (1): ReLU()
    (2): Linear(in_features=50, out_features=25, bias=True)
    (3): ReLU()
    (4): Linear(in_features=25, out_features=3, bias=True)
    (5): Softmax(dim=None)
  )
)

In [ ]:
with torch.no_grad():
    predicted_labels = model(test_data)


In [ ]:
predicted_labels

tensor([[1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        ...,
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.]], device='cuda:0')

In [ ]:
torch.sum(torch.argmax(predicted_labels, 1)==1)

tensor(0, device='cuda:0')

In [ ]:
torch.sum(torch.argmax(predicted_labels, 1)==torch.argmax(test_targets, 1))/len(predicted_labels)

tensor(0.4557, device='cuda:0')

In [ ]:
mse = mean_squared_error(test_targets.cpu().numpy(), predicted_labels.cpu().numpy())
r2 = r2_score(test_targets.cpu().numpy(), predicted_labels.cpu().numpy())
print("Mean Squared Error:", mse)
print("R2 Score:", r2)

Mean Squared Error: 0.36284863254806393
R2 Score: -0.647518903025511


In [ ]:
test_path_away ='/content/drive/MyDrive/data/Test_Data/test_away_team_statistics_df.csv'
test_path_home = '/content/drive/MyDrive/data/Test_Data/test_home_team_statistics_df.csv'
df_final= prepare_attribut_data(test_path_away, test_path_home,columns,use_player_data=True, is_test=True)
df_final_test=df_final.drop(['ID'], axis=1)

if type_pca is not None:
    df_final_test = pd.DataFrame(pca_version.transform(df_final_test), index=df_final_test.index, columns=["KPC" + str(i) for i in range(1, n_components + 1)])


In [ ]:
df_final_test.head()

,HOME_TEAM_SHOTS_TOTAL_season_sum,HOME_TEAM_SHOTS_INSIDEBOX_season_sum,HOME_TEAM_SHOTS_OFF_TARGET_season_sum,HOME_TEAM_SHOTS_ON_TARGET_season_sum,HOME_TEAM_SHOTS_OUTSIDEBOX_season_sum,HOME_TEAM_PASSES_season_sum,HOME_TEAM_SUCCESSFUL_PASSES_season_sum,HOME_TEAM_SAVES_season_sum,HOME_TEAM_CORNERS_season_sum,HOME_TEAM_FOULS_season_sum,...,HOME_PLAYER_STARTING_LINEUP_5_last_match_std,HOME_PLAYER_SUCCESSFUL_DRIBBLES_5_last_match_std,HOME_PLAYER_TACKLES_5_last_match_std,HOME_PLAYER_TOTAL_CROSSES_5_last_match_std,HOME_PLAYER_TOTAL_DUELS_5_last_match_std,HOME_PLAYER_YELLOWCARDS_5_last_match_std,HOME_PLAYER_PUNCHES_5_last_match_std,HOME_PLAYER_LONG_BALLS_5_last_match_std,HOME_PLAYER_LONG_BALLS_WON_5_last_match_std,HOME_PLAYER_SHOTS_OFF_TARGET_5_last_match_std
0,3.0,6.0,5.0,1.0,3.0,9.0,8.0,10.0,6.0,7.0,...,546.0,317.0,439.0,311.0,562.0,422.0,45.0,0.0,0.0,0.0
1,3.0,2.0,4.0,2.0,6.0,1.0,1.0,2.0,4.0,10.0,...,63.0,152.0,255.0,180.0,407.0,311.0,61.0,0.0,0.0,0.0
2,7.0,10.0,8.0,8.0,1.0,4.0,5.0,0.0,10.0,4.0,...,266.0,131.0,325.0,225.0,399.0,250.0,31.0,0.0,0.0,0.0
3,5.0,7.0,5.0,9.0,5.0,7.0,6.0,3.0,5.0,8.0,...,861.0,261.0,363.0,219.0,527.0,533.0,0.0,0.0,0.0,0.0
4,5.0,4.0,6.0,3.0,5.0,0.0,0.0,2.0,2.0,9.0,...,518.0,406.0,330.0,181.0,683.0,546.0,73.0,0.0,0.0,0.0


In [ ]:
df_test_torch=torch.from_numpy(np.array(df_final_test)).to(torch.float32)

In [ ]:
test_result=model(df_test_torch.to(device))

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py:1553: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


In [ ]:
test_result[:10]

tensor([[0.5708, 0.2129, 0.2164],
        [0.5708, 0.2129, 0.2164],
        [0.5708, 0.2129, 0.2164],
        [0.5708, 0.2129, 0.2164],
        [0.5708, 0.2129, 0.2164],
        [0.5708, 0.2129, 0.2164],
        [0.5708, 0.2129, 0.2164],
        [0.5708, 0.2129, 0.2164],
        [0.5708, 0.2129, 0.2164],
        [0.5708, 0.2129, 0.2164]], device='cuda:0', grad_fn=<SliceBackward0>)

In [ ]:
argmax=torch.argmax(test_result, 1)

In [ ]:
result = F.one_hot(argmax, num_classes=3)

In [ ]:
result_prediction = pd.DataFrame(result.cpu().numpy())


In [ ]:
result_prediction.rename(columns={'0': 'HOME_WINS', '1': 'DRAW', '2': 'AWAY_WINS'})
result_prediction.columns = ['HOME_WINS' ,'DRAW','AWAY_WINS']


In [ ]:
result_prediction.index=df_final.index

In [ ]:
test_prediction=pd.concat([df_final['ID'],result_prediction], axis=1)


In [ ]:
test_prediction

,ID,HOME_WINS,DRAW,AWAY_WINS
0,12303,1,0,0
1,12304,1,0,0
2,12305,1,0,0
3,12306,1,0,0
4,12307,1,0,0
...,...,...,...,...
25363,37666,1,0,0
25364,37667,1,0,0
25365,37668,1,0,0
25366,37669,1,0,0


In [ ]:
test_prediction.to_csv('./mlp_benchmark_submission.csv', index=False)
